In [62]:
# import keras.callbacks
from music21 import *

import re
import xml

import numpy as np
import matplotlib.pyplot as plt
import math
import keras
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dense, Dropout
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [63]:
with open('environment.txt') as f:
    env = f.readlines()
# you may also want to remove whitespace characters like \n at the end of each line
env = [x.strip() for x in env]

us = environment.UserSettings()
us['musicxmlPath'] = env[0]
us['musescoreDirectPNGPath'] = env[0]



In [64]:
# fix random seed for reproducibility
numpy.random.seed(7)

In [65]:
bachRegex = re.compile(r'.*bwv.*', re.IGNORECASE)
bd = corpus.search(bachRegex)

In [66]:


def convertSongToDataFrame(song):
    df = pd.DataFrame()
    
    currentKeySignature = None
    currentTimeSignature = None
    for measure in song.measures(0, None, collect=('TimeSignature','KeySignature')):
        if measure.keySignature:
            currentKeySignature = measure.keySignature
        if measure.timeSignature:
            currentTimeSignature = measure.timeSignature.ratioString
        for i in measure.iter:
            if type(i) is chord.Chord:
                df2 = pd.DataFrame([currentKeySignature, currentTimeSignature, roman.romanNumeralFromChord(i, currentKeySignature).figure, i.duration.quarterLength, i.offset + measure.offset])
                df = df.append(df2[0], ignore_index = True)
    return df

In [31]:
#only run if you want tocreate csv files
# import os.path
# import os
# if not os.path.exists('./data'):
#     os.makedirs("./data")
# for s in bd:
#     fname = "./data/" + s.sourcePath.split("/")[1]+'.csv'
#     if os.path.isfile(fname):
#         continue
#     df = convertSongToDataFrame(s.parse().chordify())
#     df.to_csv(fname, sep=',')
    
# #failed at bwv276.mxl

AttributeError: 'KeySignature' object has no attribute 'tonic'

In [67]:
import os
import pandas as pd

df1 = pd.read_csv('C:/Users/errpo/Desktop/projects/project folder/music_prediction/data/bwv1.6.mxl.csv')

df2 = pd.read_csv('C:/Users/errpo/Desktop/projects/project folder/music_prediction/data/bwv2.6.mxl.csv')

In [68]:
class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))

In [69]:
merged = df1.merge(df2, left_index=True, right_index=True, how='inner').dropna()
data = merged.values

examples = 2
y_examples = 2
nb_samples = len(data) - examples - y_examples

In [70]:
# input - 2 features
input_list = [np.expand_dims(np.atleast_2d(data[i:examples+i,:]), axis=0) for i in range(nb_samples)]
input_list
input_mat = np.concatenate(input_list, axis=0)


In [71]:
target_list = [np.atleast_2d(data[i+examples:examples+i+y_examples,0]) for i in range(nb_samples)]
target_mat = np.concatenate(target_list, axis=0)

In [72]:
# set up model
trials = input_mat.shape[0]
features = input_mat.shape[2]
hidden = 64
model = Sequential()
model.add(LSTM(hidden, input_shape=(examples, features)))
model.add(Dropout(.2))
model.add(Dense(y_examples))
model.add(Activation('linear'))
model.compile(loss='mse', optimizer='rmsprop')

# Train
history = LossHistory()
model.fit(input_mat, target_mat, nb_epoch=100, batch_size=400, callbacks=[history])

c:\users\errpo\anaconda3\envs\music\lib\site-packages\keras\models.py:939: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/100


ValueError: could not convert string to float: 'bVI+6'